In [ ]:
import pandas as pd
file_path = r"C:\Users\VIVEK YADAV\Desktop\nextbill\bank_data_for_ai\NPIA\BankStatement\SBI Statement (April 1-4-23 to 30-4-23).xlsx"
df = pd.read_excel(file_path, header=19, usecols="A:I", nrows=42)

df.columns = df.columns.str.lower().str.replace(' ', '_')
df['credit'] = df['credit'].replace('', 0).fillna(0)
df['debit'] = df['debit'].replace('', 0).fillna(0)



import pandas as pd
import time
import json
from openai import AzureOpenAI

AZURE_OPENAI_KEY = ""
AZURE_OPENAI_ENDPOINT = ""
AZURE_DEPLOYMENT_NAME = ""  # e.g., gpt-4o-mini
AZURE_API_VERSION = ""  # or whatever you're using

# Initialize client
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=AZURE_API_VERSION,
)

# Define GPT function
def call_gpt(narration, direction):
    prompt = f"""
You are a financial data extractor. Given a bank transaction narration, extract the following:

- sender_or_reciever_info: if transaction is credit/deposit, sender name else reciever name
- transaction_type: method of transaction like NEFT, UPI, IMPS, etc.
- currency: if foreign currency involved, name it (e.g. USD, EUR)
- exchange_rate: if mentioned, give exchange rate as number

The transaction direction is '{direction}' (either "credit" or "debit"). 
You can determine transaction direction by checking if credit/deposit has value or debit/withdrawl has value

Narration:
\"{narration}\"

Respond only with a valid JSON dictionary like:
{{
  "sender_or_reciever_info": "...",
  "transaction_type": "...",
  "currency": "...",
  "exchange_rate": ...
}}
"""
    try:
        response = client.chat.completions.create(
            model=AZURE_DEPLOYMENT_NAME,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=200,
        )
        content = response.choices[0].message.content.strip()
        return json.loads(content)
    except Exception as e:
        print(f"Error processing: {narration[:30]} -> {e}")
        return {
            "sender_or_reciever_info": "nan",
            "transaction_type": "nan",
            "currency": "nan",
            "exchange_rate": "nan"
        }

# Process rows and collect outputs
results = []
for i, row in df.iterrows():
    narration = row["description"]
    #direction = "credit" if row["credit"] > 0 else "debit"
    direction = "credit" if isinstance(row["credit"], int) else "debit"
    
    result = call_gpt(narration, direction)
    results.append(result)
    
    # Optional: sleep to avoid rate limits
    time.sleep(1)

# Convert extracted features into a DataFrame
extracted_df = pd.DataFrame(results)

# Combine with original data
combined_df = pd.concat([df.reset_index(drop=True), extracted_df], axis=1)

# Save output
combined_df.to_csv("NPI_feature_extraction.csv", index=False)
print("✅ Extraction complete. File saved as Bank_Statement_with_Extracted_Features.csv.")